# Task 1

In [1]:
# importing libaries
import tensorflow as tf
import tarfile
import gzip
import shutil
import librosa
import numpy as np
import os

In [2]:
# The following code was copied from the textbook:
# Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow
# I modified the head of the model to make it a predict
# the amplitude of the audio signal that immediately follows
# the 5 amplitudes that precede it (the input)
wavenet_model = tf.keras.Sequential()
wavenet_model.add(tf.keras.layers.Input(shape=[None, 5]))
for rate in (1, 2, 4, 8) * 2:
  wavenet_model.add(tf.keras.layers.Conv1D(
  filters=32, kernel_size=2, padding="causal", activation="relu", dilation_rate=rate))
wavenet_model.add(tf.keras.layers.Conv1D(filters=1, kernel_size=1))

# Task 2

I downloaded train_normal.tar.gz, test_normal.tar.gz, and test_anomaly.tar.gz, placing them into my working directory. The following are useful functions for processing these files.

In [3]:
# extracts file, placing it into directory with same name
# as file without the file extension
def extract(file):
  with gzip.open(file, 'rb') as inFile:
      with tarfile.open(fileobj=inFile, mode='r') as tar:
          tar.extractall(path=file.split('.')[0])

In [4]:
# recursively deletes a directory
def delete(directory):
  shutil.rmtree(directory)

In [5]:
# returns list of path for all files
# in a directory
def paths(directory):
  result = []
  for fileName in os.listdir(directory):
    result.append(directory + "/" + fileName)
  return result

The following function generates our data from a given file. It will include the amplitude for 5 frames with the 6th frame being the output or label.

In [16]:
# returns list of training data for
# a file
def generate(file_path):
  audio_data, sample_rate = librosa.load(file_path, sr=None)
  audio_data = librosa.util.normalize(audio_data)
  audio_data = audio_data.flatten()
  data = []
  labels = []
  for i in range(0, len(audio_data) - 3, 6):
    data.append(audio_data[i:i+5])
    labels.append(audio_data[i+5])
  return (np.array(data), np.array(labels))

In [7]:
extract("train_normal.tar.gz")
extract("test_normal.tar.gz")
extract("test_anomaly.tar.gz")

In [8]:
train_files = paths("train_normal")
test_files = paths("test_normal")

In [10]:
# quartering the amount of training data
# due to limited RAM capabilities
train_files = train_files[:len(train_files)//4]
test_files = test_files[:len(test_files)//4]

In [58]:
# generates training data/labels
x_train = []
y_train = []
for f in train_files:
  res = generate(f)
  x_train.append(res[0])
  y_train.append(res[1])
temp = []
for x in x_train:
    for y in x:
        temp.append(y)
x_train = np.array(temp)
y_train = np.array(y_train).ravel()

In [59]:
# generates testing data/labels
x_test = []
y_test = []
for f in test_files:
  res = generate(f)
  x_test.append(res[0])
  y_test.append(res[1])
temp = []
for x in x_test:
    for y in x:
        temp.append(y)
x_test = np.array(temp)
y_test = np.array(y_test).ravel()

# Task 3

In [60]:
wavenet_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [69]:
print(len(x_train[0]))
print(len(y_train))
print(len(x_test))
print(len(y_test))

5


TypeError: 'int' object is not subscriptable

In [65]:
batch_size = 32
#x_train = np.reshape(x_train, (batch_size, 1, 5))
#x_test = np.reshape(x_test, (batch_size, 1, 5))
#y_train = np.reshape(y_train, (batch_size, 1, 1))
#y_test = np.reshape(y_test, (batch_size, 1, 1))

In [66]:
wavenet_model.fit(x_train, y_train, batch_size=batch_size, epochs=10, validation_data=(x_test, y_test), verbose=1)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\saumy\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\saumy\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\saumy\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\saumy\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\saumy\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   